In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
## Loading images and labels
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

## Resizing images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

## Transforming labels to correct format
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

In [ ]:
## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds[0].shape)
base_model.trainable = False ## Not trainable weights

## Preprocessing input
train_ds = preprocess_input(train_ds) 
test_ds = preprocess_input(test_ds)

base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

Epoch 1/50
65/65 [==============================] - 17s 130ms/step - loss: 3.1504 - accuracy: 0.4433 - val_loss: 1.3863 - val_accuracy: 0.5175
Epoch 2/50
65/65 [==============================] - 7s 102ms/step - loss: 1.0690 - accuracy: 0.6307 - val_loss: 1.2888 - val_accuracy: 0.5681
Epoch 3/50
65/65 [==============================] - 7s 103ms/step - loss: 0.6882 - accuracy: 0.7538 - val_loss: 1.1978 - val_accuracy: 0.6381
Epoch 4/50
65/65 [==============================] - 7s 104ms/step - loss: 0.4782 - accuracy: 0.8321 - val_loss: 1.2657 - val_accuracy: 0.6265
Epoch 5/50
65/65 [==============================] - 7s 103ms/step - loss: 0.3686 - accuracy: 0.8706 - val_loss: 1.3212 - val_accuracy: 0.6342
Epoch 6/50
65/65 [==============================] - 8s 122ms/step - loss: 0.3098 - accuracy: 0.8910 - val_loss: 1.3523 - val_accuracy: 0.6634
Epoch 7/50
65/65 [==============================] - 7s 102ms/step - loss: 0.2689 - accuracy: 0.9041 - val_loss: 1.2748 - val_accuracy: 0.6809
Epoch